In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
base_url = 'https://247sports.com/'
url = base_url + "Season/2021-Football/CollegeTeamTalentComposite/?Conference=";
conferences = ['ACC','Big-12','Big-Ten','Pac-12','SEC','AAC']

In [10]:
# DATA
data = get_teams_general_info()

In [5]:
def parse_team_info(soup):
    a = soup.find('a', {'class':'rankings-page__name-link'})
    teamName = a.string.strip()
    team_breakdown_url = a['href'] if a.has_attr('href') else None
    logo_container = soup.find('img')
    logo_url = logo_container['data-src'] if logo_container.has_attr('data-src') else None
    if logo_url is not None:
        logo_url = logo_url.split('?')[0]
    
    team_data = {'name':teamName,'team_url' : team_breakdown_url, 'logo_url' : logo_url}
    return team_data

def get_teams_general_info():
    data = {}
    for conference in conferences:
        conference_url = url + conference
        html = requests.get(conference_url, headers={"User-Agent": "Requests"}).content
        soup = BeautifulSoup(html, 'html.parser')
        team_info_containers = soup.find_all('li', {'class' : 'rankings-page__list-item'})
        for team_container in team_info_containers:
            team_data = parse_team_info(team_container)
            team_data['conference'] = conference
            data[team_data['name']] = team_data
    return data

In [11]:
def get_team_talent_data(team_data, team_name):
    team_url = base_url + team_data['team_url']
    html = requests.get(team_url, headers={"User-Agent": "Requests"}).content
    soup = BeautifulSoup(html, 'html.parser')
    table_body = soup.find('tbody')
    # extract team members
    containers = table_body.find_all('td')
    roster = []
    for container in containers:
        a = container.find('a')
        name = a.string if a else container.find('span').string
        roster.append(name)
    
    players = []
    # extract talent info
    stats_container = soup.find('div', {'class' : 'scroll-table-container'})
    table_body = stats_container.find('tbody')
    rows = table_body.find_all('tr')
    if len(rows) != len(roster):
        print("##### error: mismach roster and stats")
    else:
        for i in range(len(rows)):
            name = roster[i]
            row = rows[i]
            stats = row.find_all('td')
            position = stats[1].string
            height = stats[2].string
            weight = stats[3].string
            year = stats[4].string
            stars = row.find_all('span', {'class':'icon-starsolid'})
            player = {'name':name, 'team': team_name, 'position' : position, 'height' : height, 'weight' : weight, 'year' : year, 'stars' : len(stars)}
            players.append(player)
    return players
    

In [12]:
oklahoma = data['Oklahoma']
players = get_team_talent_data(oklahoma, 'Oklahoma')

In [18]:
playerDB = []
for team_name in data:
    print("fetching data for: " + team_name)
    players = get_team_talent_data(data[team_name], team_name)
    playerDB.extend(players)
    
    

fetching data for: Clemson
fetching data for: Miami
fetching data for: North Carolina
fetching data for: Florida State
fetching data for: Georgia Tech
fetching data for: NC State


KeyboardInterrupt: 